In [9]:
import itertools
import random
import pandas
import time
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.legend_handler import HandlerLine2D

import sys
sys.path.append('.')
import RM_helper
import RM_compare
import RM_exact
import RM_approx
import RM_ADP

In [68]:
PRICE_LIMITS = [150, 250]

def generate_network(n_spokes, demand_model):
    """Generates a network using the given number of spokes, and the demand model, with random prices, and arrival rates
    of itineraries. Currently only supports 1 fare class per itinerary. """
    resources = [] # records flight legs names
    itineraries = [] # records names and (revenue, arrival rate) pairs of fare classes of itineraries
    hub_name = 'hub'
    spoke_names = []
    
    # produce flight legs (single-direction)
    for i in range(n_spokes):
        spoke_name = chr(65 + i)
        spoke_names.append(spoke_name)
        resources.append(spoke_name + '-' + hub_name)
    
    # produce single-leg itineraries
    single_legs = resources[:]
    single_legs += reverse_itinerary(resources)
    
    # produce double-leg itineraries
    double_legs = []
    two_spoke_pairs = list(itertools.combinations(''.join(spoke_names), 2))
    for pair in two_spoke_pairs:
        iti = '-'.join([pair[0], hub_name, pair[1]])
        double_legs.append(iti)
    
    double_legs += reverse_itinerary(double_legs)
    
    # produce double-leg itineraries, between the hub and the same spoke, i.e. round-trips between spoke and hub
    round_legs = []
    for spoke in spoke_names:
        round_legs.append('-'.join([spoke, hub_name, spoke]))
    
    # aggregate all itineraries, and randomly generate the price and arrival rate
    itineraries += single_legs + double_legs + round_legs
    f = len(itineraries)
    demands = generate_random_arrival_rate(f, 0.3, demand_model)
    for i in range(f):
        full_iti = [itineraries[i]]
        arrival_rate = demands[i]
        price = generate_random_price(itineraries[i])
        full_iti.append([(price, arrival_rate)])
        itineraries[i] = full_iti
    return resources, itineraries
    
def reverse_itinerary(itinerary_names):
    """helper func: given a list of itinerary names, generate a list of reversed itineraries for them. """
    reversed_itineraries = []
    for itinerary in itinerary_names:
        nodes = itinerary.split('-')
        nodes.reverse()
        reversed_name = '-'.join(nodes)
        reversed_itineraries.append(reversed_name)
    return reversed_itineraries

def generate_random_arrival_rate(n, total_sum, demand_model):
    """helper func: generate n random values in [0,1] and normalize them so that their sum is equal to total_sum."""
    if demand_model == 1:
        # constant arrival rates for classes over time
        M = sys.maxsize
        x = random.sample(range(M), n - 1)
        x.insert(0, 0)
        x.append(M)
        x.sort()
        y = [x[i + 1] - x[i] for i in range(n)]
        unit_simplex = [y_i / (1/total_sum * M) for y_i in y]
        return unit_simplex
    else:
        print("TODO:implement model 2")

def generate_random_price(itinerary_name):
    """helper func: generate a random price for the given itinerary, limit depends on how many flight legs it uses."""
    leg_num = itinerary_name.count('-')
    price = random.randint(50, PRICE_LIMITS[leg_num-1])
    return price

resources, itineraries = generate_network(3, 1)
RM_helper.extract_legs_info(itineraries, resources)

([[1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0],
  [0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0],
  [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1],
  [0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0],
  [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0],
  [0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1]],
 [['A-hub,1', 136, 0.015189303062234503],
  ['B-hub,1', 64, 0.011254552884196198],
  ['C-hub,1', 71, 0.05055641395361463],
  ['hub-A,1', 59, 0.014519949081951273],
  ['hub-B,1', 114, 0.029522310842602498],
  ['hub-C,1', 63, 0.013761512528204777],
  ['A-hub-B,1', 80, 0.012297494696835953],
  ['A-hub-C,1', 139, 0.004616994228393384],
  ['B-hub-C,1', 154, 0.006180509636767171],
  ['B-hub-A,1', 102, 0.00010985010143045068],
  ['C-hub-A,1', 181, 0.012781348349624438],
  ['C-hub-B,1', 59, 0.01875903162723695],
  ['A-hub-A,1', 60, 0.06147251694356827],
  ['B-hub-B,1', 205, 0.047610643608380306],
  ['C-hub-C,1', 55, 0.0013675684549591728]])

In [19]:
def compare_EMSR_b_with_exact_single_static(pros, cap, iterations):
    """Compare the EMSR-b method, with single-static DP model."""
    products, demands,_ = RM_helper.sort_product_demands(pros)
    
    diff_percents = []
    
    exact = RM_exact.Single_RM_static(products, demands, cap)
    exact_bid_prices = exact.get_bid_prices()
    exact_protection_levels = exact.get_protection_levels()
    
    heuri = RM_approx.Single_EMSR(products, demands, cap)
    heuri_protection_levels = heuri.get_protection_levels()

    bid_prices = [exact_bid_prices]
    protection_levels = [exact_protection_levels, heuri_protection_levels]
    
    exact_revs_diff = []
    exact_LF_diff = []
    exact_heuri_revs_diff = []
    exact_heuri_LF_diff = []

    results = [exact_protection_levels]
    for i in range(iterations):
        requests = RM_helper.sample_single_static_demands(demands)
        bp_result = RM_compare.simulate_single_static_bidprices_control(bid_prices, products, demands, cap, requests)
        pl_result = RM_compare.simulate_single_static_protectionlevel_control(protection_levels, products, demands, \
                                                                               cap, requests)
        
        exact_pl_rev = pl_result[0][0]
        exact_pl_LF = pl_result[0][1]
    
        exact_revs_diff.append(round((exact_pl_rev - bp_result[0][0])/ exact_pl_rev, 5))
        exact_LF_diff.append(round((exact_pl_LF - bp_result[0][1])/ exact_pl_LF, 5))
        exact_heuri_revs_diff.append(round((exact_pl_rev - pl_result[1][0])/exact_pl_rev, 5))
        exact_heuri_LF_diff.append(round((exact_pl_LF - pl_result[1][1]) / exact_pl_LF, 5))

    results+= [np.mean(exact_revs_diff) * 100, np.mean(exact_LF_diff) * 100, heuri_protection_levels, 
               np.mean(exact_heuri_revs_diff) * 100, np.mean(exact_heuri_LF_diff) * 100]
    return results

def visualize_perf_EMSR_b(products, cap_lb, cap_ub, cap_interval, iterations):
    """Visualize the performance of EMSR-b method, against single-static DP model."""
    capacities = [c for c in range(cap_lb, cap_ub + 1, cap_interval)]
    col_titles = ["exact-protection_levels", "mean-diff_exact %", "mean_diff_exact_LF %", "EMSR-b-protection_levels", \
                  "mean-diff_pl %", "mean-diff_pl_LF %"]

    table_data = []
    
    for cap in capacities:
        result= compare_EMSR_b_with_exact_single_static(products, cap, iterations)
        
        table_data.append(result)
    
    print(pandas.DataFrame(table_data, capacities, col_titles))
    return table_data

pros = [[1, 1050,(17.3, 5.8)], [2, 567, (45.1, 15.0)], [3, 534, (39.6, 13.2)], [4,520,(34.0, 11.3)]]
# pros = [[1, 1050,(17.3, 5.8)], [2, 950, (45.1, 15.0)], [3, 699, (39.6, 13.2)], [4,520,(34.0, 11.3)]]
cap_lb = 80
cap_ub = 160
cap_interval = 10
iteration = 100

# data = visualize_perf_EMSR_b(pros, cap_lb, cap_ub,cap_interval,iteration)
# exact_revs = [d[1] for d in data]
# exact_LF = [d[2] for d in data]

# plt.clf()
# x= np.linspace(cap_lb, cap_ub, (cap_ub - cap_lb) / cap_interval + 1)
# plt.plot(x, exact_revs, linestyle='dashed', marker='s', label='Revenue Difference')
# plt.plot(x, exact_LF, linestyle='dashed', marker = 'o', label='Load Factor Difference')
    
# plt.legend()
# plt.ylabel('Bid-price vs Protection-level Control')
# plt.xlabel('Resource Capacity')
# # plt.show()
# plt.savefig('single_static_exact_diff')


# exact_heuri_revs = [d[4] for d in data]
# exact_heuri_LF = [d[5] for d in data]
# plt.clf()
# x= np.linspace(cap_lb, cap_ub, (cap_ub - cap_lb) / cap_interval + 1)
# plt.plot(x, exact_heuri_revs, linestyle='dashed', marker='s', label='Revenue Difference')
# plt.plot(x, exact_heuri_LF, linestyle='dashed', marker = 'o', label='Load Factor Difference')
    
# plt.legend()
# plt.ylabel('Exact vs EMSR-b Protection-levels Control')
# plt.xlabel('Resource Capacity')
# # plt.show()
# plt.savefig('single_static_diff')



In [ ]:
def compare_ADP_LP_w_DLP(T_lb, T_ub, T_interval):
    # 3 spoke situation 
    
    
    